In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

# Load training dataset
train_df = pd.read_csv('train.csv')

# Define a function for text preprocessing
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

# Preprocess text data
train_df['Text'] = train_df['Text'].apply(preprocess_text)

# Split dataset into features and labels
X_train = train_df['Text']
y_train = train_df['Emotion']

# Define classifiers
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Linear SVC': LinearSVC(),
}

# Train classifiers
trained_classifiers = {}
for name, clf in classifiers.items():
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', clf)
    ])
    pipeline.fit(X_train, y_train)
    trained_classifiers[name] = pipeline

# Check if training is successful
print("Training completed successfully.")

# Function to predict emotion
def predict_emotion(text):
    text = preprocess_text(text)
    nb_prediction = trained_classifiers['Naive Bayes'].predict([text])[0]  # Predict with Naive Bayes
    svc_prediction = trained_classifiers['Linear SVC'].predict([text])[0]  # Predict with Linear SVC
    return nb_prediction, svc_prediction

# Function to handle button click
def analyze_text():
    text = text_entry.get('1.0', tk.END).strip()
    true_emotion = true_emotion_var.get()

    if text and true_emotion:
        nb_prediction, svc_prediction = predict_emotion(text)
        result_label.config(text=f'Naive Bayes Prediction: {nb_prediction}\nLinear SVC Prediction: {svc_prediction}')
    else:
        messagebox.showwarning('Warning', 'Please enter text and select true emotion.')

# Function to handle Save to CSV button click
def save_to_csv():
    text = text_entry.get('1.0', tk.END).strip()
    true_emotion = true_emotion_var.get()

    if text and true_emotion:
        with open('train.csv', 'a') as f:
            f.write(f'{text},{true_emotion}\n')
        messagebox.showinfo('Success', 'Data saved to train.csv.')
    else:
        messagebox.showwarning('Warning', 'Please enter text and select true emotion.')

# Create GUI window
root = tk.Tk()
root.title('Emotion Detection')

# Create text entry widget
text_label = tk.Label(root, text='Enter Text:')
text_label.pack()
text_entry = tk.Text(root, height=5, width=50)
text_entry.pack()

# Create dropdown for selecting true emotion
true_emotion_label = tk.Label(root, text='Select True Emotion:')
true_emotion_label.pack()
true_emotion_var = tk.StringVar(root)
true_emotion_dropdown = ttk.Combobox(root, textvariable=true_emotion_var, values=['sadness', 'anger', 'love', 'surprise', 'joy', 'fear'])
true_emotion_dropdown.pack()

# Create button to analyze text
analyze_button = tk.Button(root, text='Analyze Text', command=analyze_text)
analyze_button.pack()

# Display predicted emotion
result_label = tk.Label(root, text='')
result_label.pack()

# Create button to save data to CSV
save_button = tk.Button(root, text='Save to CSV', command=save_to_csv)
save_button.pack()

root.mainloop()


ModuleNotFoundError: No module named 'seaborn'

Testing Part

In [ ]:
import pandas as pd

# Load test dataset
test_df = pd.read_csv('test.csv')

# Preprocess text data
test_df['Text'] = test_df['Text'].apply(preprocess_text)

# Predict emotions using trained classifiers
for name, clf in trained_classifiers.items():
    # Make predictions
    y_pred = clf.predict(test_df['Text'])
    # Add predictions to test dataframe
    test_df[name] = y_pred

# Write predictions to test.csv
test_df.to_csv('test.csv', index=False)

# Check if predictions are written successfully
print("Predictions written to test.csv.")


Graphs and Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix

# Load test dataset with predictions
test_df = pd.read_csv('test.csv')

# Define function to plot bar graphs
def plot_bar_graph(data, xlabel, ylabel, title):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=data.index, y=data.values, palette='Set2')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

# Define function to plot confusion matrix with custom tick labels
def plot_confusion_matrix(matrix, labels, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(matrix, annot=True, fmt='d', cmap='YlGnBu', cbar=False, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Emotion')
    plt.ylabel('True Emotion')
    plt.title(title)
    plt.show()

# Calculate accuracy for each classifier
accuracy_results = {}
for name, _ in trained_classifiers.items():
    accuracy = accuracy_score(test_df['True_emotion'], test_df[name])
    accuracy_results[name] = accuracy

# Plot accuracy for each classifier
plot_bar_graph(pd.Series(accuracy_results), xlabel='Classifier', ylabel='Accuracy', title='Accuracy for Each Classifier')

# Generate confusion matrix for each classifier
confusion_matrices = {}
for name, _ in trained_classifiers.items():
    confusion_matrices[name] = confusion_matrix(test_df['True_emotion'], test_df[name])

# Get unique emotion labels
emotion_labels = test_df['True_emotion'].unique()

# Plot confusion matrix for each classifier with custom tick labels
for name, matrix in confusion_matrices.items():
    plot_confusion_matrix(matrix, labels=emotion_labels, title=f'Confusion Matrix for {name}')
